In [9]:
import requests
import numpy as np
import pandas as pd
import time

In [21]:
tickers = ["AAPL", "IBM", "HSY"]


def find_fvg(stocks_data):
    """
    stocks_data: list of tuples like [(symbol, np_array_of_ohlc_and_volume), ...]
                 each np_array shape = (num_candles, at least 6) where:
                 [0]=open, [1]=high, [2]=low, [3]=close, [4]=volume, [5]=20-day avg volume
    """
    for symbol, data in stocks_data:
        # Loop through 3-candle windows
        for i in range(len(data) - 2):
            first_h = data[i][1]
            first_l = data[i][2]
            second_v = data[i+1][4]
            second_avg_v = data[i+1][5]
            third_h = data[i+2][1]
            third_l = data[i+2][2]

            # Only flag if 2nd candle has ≥ 2× 20-day avg volume
            if second_v >= 2 * second_avg_v:
                # Bullish FVG
                if first_h < third_l:
                    print(f"Stock {symbol}, Candle {i+2}, Bullish FVG (2nd candle volume {second_v:.0f} ≥ 2× avg {second_avg_v:.0f})")
                # Bearish FVG
                elif first_l > third_h:
                    print(f"Stock {symbol}, Candle {i+2}, Bearish FVG (2nd candle volume {second_v:.0f} ≥ 2× avg {second_avg_v:.0f})")

for i in range(len(tickers)):
    stock = tickers[i]
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock}&apikey=L5918IH11TGFFE2L'
    r = requests.get(url)
    data = r.json()

    time.sleep(0.7)
    
    # Extract the time series
    data = data["Time Series (Daily)"]
    
    # Storage
    full = []             # list of np arrays (one per candle)
    opens = np.array([])  # full list of opens
    highs = np.array([])
    lows = np.array([])
    closes = np.array([])
    volumes = np.array([])
    
    
    # Loop through candles
    for date, candle in data.items():
        o = float(candle["1. open"])
        h = float(candle["2. high"])
        l = float(candle["3. low"])
        c = float(candle["4. close"])
        v = float(candle["5. volume"])
    
        # Print each candle
        #print(f"{date}  open={o}  high={h}  low={l}  close={c}  volume={v}")
    
        # Store the candle as a NumPy array
        arr = np.array([o, h, l, c, v])
        full.append(arr)
    
        # Build full lists
        opens = np.append(opens, o)
        highs = np.append(highs, h)
        lows = np.append(lows, l)
        closes = np.append(closes, c)
        volumes = np.append(volumes, v)
    
    # Assuming full is your Python list of arrays from before
    full_np = np.array(full)  # converts list of arrays into a 2D NumPy array
    
    #print(full_np.shape)  # should be (number_of_days, 5) for o,h,l,c,v
    #print(full_np[:5])    # print first 5 rows
    
    
    # Convert to DataFrame for easier calculation
    df = pd.DataFrame(data).T  # transpose so dates are rows
    df = df.sort_index()  # ensure dates are ascending
    df = df.astype(float)
    df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    }, inplace=True)
    
    df['avg_volume'] = df['volume'].rolling(window=20).mean()
    
    # Calculate Average Daily Range (ADR)
    df['adr'] = df['high'] - df['low']
    
    # Calculate Moving Averages
    df['ma10'] = df['close'].rolling(window=10).mean()
    df['ma20'] = df['close'].rolling(window=20).mean()
    df['ma50'] = df['close'].rolling(window=50).mean()
    
    # Calculate Average Volume (simple rolling average over 20 days)
    
    # Calculate VWAP (cumulative typical price * volume / cumulative volume)
    tp = (df['high'] + df['low'] + df['close']) / 3
    df['vwap'] = (tp * df['volume']).cumsum() / df['volume'].cumsum()
    
    # Calculate RSI (14-day default)
    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    opens = df['open'].to_numpy()
    highs = df['high'].to_numpy()
    lows = df['low'].to_numpy()
    closes = df['close'].to_numpy()
    volumes = df['volume'].to_numpy()
    adr = df['adr'].to_numpy()
    vwap = df['vwap'].to_numpy()
    avg_volume = df['avg_volume'].to_numpy()
    ma10 = df['ma10'].to_numpy()
    ma20 = df['ma20'].to_numpy()
    ma50 = df['ma50'].to_numpy()
    rsi = df['rsi'].to_numpy()
    
    ma10 = np.nan_to_num(ma10, nan=0)
    rsi = np.nan_to_num(rsi, nan=0)
    avg_volume = np.nan_to_num(ma10, nan=0)
    ma20 = np.nan_to_num(rsi, nan=0)
    ma50 = np.nan_to_num(ma10, nan=0)
    vwap = np.nan_to_num(rsi, nan=0)
    
    #converts everything into 1, 2D array that is a numpy array.
    extras_np = np.column_stack([
        df['open'].to_numpy(),
        df['high'].to_numpy(),
        df['low'].to_numpy(),
        df['close'].to_numpy(),
        df['volume'].to_numpy(),
        df['avg_volume'].to_numpy(),
        df['adr'].to_numpy(),
        df['vwap'].to_numpy(),
        df['ma10'].to_numpy(),
        df['ma20'].to_numpy(),
        df['ma50'].to_numpy(),
        df['rsi'].to_numpy()
    ])
    
    #print(extras_np.shape)  # (number_of_days, 12)
    #print(extras_np[:5])
    
    #FINDING FVGS
    '''def find_fvg(stocks_data):
        for symbol, ohlc in stocks_data:
            # Loop through candles in 3-candle windows
            for i in range(len(ohlc) - 2):
                first_h = ohlc[i][1]   # high of 1st candle
                first_l = ohlc[i][2]   # low of 1st candle
                third_h = ohlc[i+2][1] # high of 3rd candle
                third_l = ohlc[i+2][2] # low of 3rd candle
    
                # Bullish FVG
                if first_h < third_l:
                    print(f"Stock {symbol}, Candle {i+2}, Bullish FVG")  # index of 3rd candle
    
                # Bearish FVG
                elif first_l > third_h:
                    print(f"Stock {symbol}, Candle {i+2}, Bearish FVG")  # index of 3rd candle'''
    
    
    ohlc_only = extras_np[:, :6]  # o,h,l,c columns
    stocks = [(stock, ohlc_only)]
    find_fvg(stocks)

KeyError: 'Time Series (Daily)'

In [20]:
stock = tickers[i]
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=HSY&apikey=L5918IH11TGFFE2L'
r = requests.get(url)
data = r.json()

print(data)

{'Note': 'We have detected your API key as L5918IH11TGFFE2L and our standard API rate limit is 25 requests per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}
